Question1 : Block matching 
Use of SSD to establish best fit

Common Segment: loading the image and creating required placeholder/vars ..etc ...

In [1]:
import os
import numpy as np
import cv2
import math
import sys

In [2]:
left_img = cv2.imread('view1.png', 0)  #read it as a grayscale image
right_img = cv2.imread('view5.png', 0)
left_depth = cv2.imread('disp1.png',0)
right_depth = cv2.imread('disp5.png',0)

In [3]:
def getSample(windowDim,xPos,yPos,srcImg,test=False,direction=0):
    """This function returns a nxn dimension subsample centred around u,v from given source"""
    deltaIndex=int((windowDim-1)/2)
    srcDim = srcImg.shape
    if(test==True):
        if(srcDim[0]==windowDim):
            if(direction==0):
                if((yPos + deltaIndex) >= srcDim[1]):
                    return False
                elif((xPos - deltaIndex) < 0):
                    return False
                else:
                    return True
            else:
                if((yPos - deltaIndex) < 1):
                    return False
                elif((xPos - deltaIndex) < 0):
                    return False
                else:
                    return True
        else:
            #print(srcDim)
            if((xPos+deltaIndex)>=srcDim[0]):
                return False
            elif((xPos-deltaIndex)<0):
                return False
            elif((yPos+deltaIndex)>=srcDim[1]):
                return False
            elif((yPos-deltaIndex)<0):
                return False
            else:
                return True
    else:
        if(srcDim[0]==windowDim):
            """ it is for one row comparison"""
            x = srcImg[:,yPos-deltaIndex:yPos+deltaIndex+1]
        else:
            x = srcImg[xPos-deltaIndex:xPos+deltaIndex+1,yPos-deltaIndex:yPos+deltaIndex+1]
        #if(x.shape[0]!=x.shape[1]):
        #    print(xPos,yPos,srcDim)
        return x

In [4]:
def getSSD(npArray1, npArray2):
    """This function returns the Sum of Square Difference for two given blocks"""
    npArray1 = npArray1.astype(np.int_)
    npArray2 = npArray2.astype(np.int_)
    diff = npArray1 - npArray2
    diffSq = np.square(diff)
    diffSq = diffSq.flatten()
    SSD = np.sum(diffSq)
    return SSD
    

In [5]:
def getDisparityValue(refSample,u,v,varSampleRow,limit,l2r=True,dim=3):
    """This function returns the disparity value given
        reference Sample, position of the reference sample, limit, left to right and
        shape of the original image(Complete)"""
    leastSSD = (sys.maxint,(u,v))
    SSDList = []
    #dimComplete = completeImgShape[1]
    for i in range(0,limit):
        if(l2r==True):
            if (getSample(windowDim=dim, xPos=u, yPos= (v + i), srcImg=varSampleRow, test=True) == False):
                break
            varSubSample=getSample(windowDim=dim,xPos=u,yPos=(v+i),srcImg=varSampleRow)
        else:
            if (getSample(windowDim=dim,xPos= u,yPos= (v-i), srcImg=varSampleRow, test=True, direction=1) == False):
                break
            varSubSample=getSample(windowDim=dim,xPos=u,yPos=(v-i),srcImg=varSampleRow)
            if(refSample.shape!=varSubSample.shape):
                print(varSubSample.shape,i,u,v)
        varSSD = getSSD(refSample,varSubSample)
        SSDList.append(int(varSSD))
        if(varSSD<leastSSD[0]):
            #leastSSD[0] = varSSD
            if(l2r==True):
                leastSSD = (varSSD,(u,v+i))
            else:
                leastSSD = (varSSD,(u,v-i))
        if(varSSD==0):
            break
    #leastSSDPos = leastSSD[1]
    #disparity = int(math.fabs(v-leastSSDPos[1]))
    if(len(SSDList) >0):
        indexLeastSSD = np.argmin(SSDList)
    else:
        indexLeastSSD = int(0)
    disparity = indexLeastSSD
    return disparity

In [6]:
from matplotlib import pyplot as plt
def plot_me(img):
    plt.imshow(img, cmap = 'gray', interpolation = 'bicubic')
    plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.show()

In [7]:
#looping through the reference image to build the disparityMatrix
def getDisparityMatrix(refImage,secondImage,wDim,wLimit=100,l2r=True):
    #numRows = refImage.shape[0]
    delta = int((wDim-1)/2)
    xpad = ((delta,delta),(delta,delta))
    ref_imgPad = np.pad(refImage,xpad,mode='constant',constant_values=0)
    second_imgPad = np.pad(secondImage,xpad,mode='constant',constant_values=0)
    DisparityMatrix = np.zeros(refImage.shape,dtype=np.int_)
    DisparityMatrix = np.pad(DisparityMatrix,xpad,mode='constant',constant_values=0)
    for i in range(delta,ref_imgPad.shape[0]-delta):
        """ sr - right side img subset .. height N, width img_width"""
        sR_set = second_imgPad[i - delta:i + delta + 1, :]
        for j in range(delta,ref_imgPad.shape[1]-delta):
            checkTest = getSample(windowDim=wDim,xPos=i,yPos=j,srcImg=ref_imgPad,test=True)
            if(checkTest==True):
                sL=getSample(windowDim=wDim,xPos=i,yPos=j,srcImg=ref_imgPad,test=False)
                value = int(getDisparityValue(refSample=sL,u=i,v=j,varSampleRow=sR_set,limit=wLimit,l2r=l2r,dim=wDim))
                DisparityMatrix[i,j]=value
            else:
                #return DisparityMatrix
                pass
    return DisparityMatrix

In [8]:
#plot_me(DM)

In [9]:
#plot_me(DM_left)

In [10]:
#plot_me(DM_right)

First Segment:Generate disparity maps for 3x3 and 9x9 window sizes

In [11]:
windowSize=3
delta = int((windowSize-1)/2)
#xpad = ((delta,delta),(delta,delta))
#DM = np.pad(DisparityMatrix,xpad,mode='constant',constant_values=0)
DM_left_n3 = getDisparityMatrix(refImage=left_img,secondImage=right_img,wDim=windowSize,wLimit=75,l2r=False)
DM_right_n3 = getDisparityMatrix(refImage=right_img,secondImage=left_img,wDim=windowSize,wLimit=75,l2r=True)

In [12]:
DM_left_n3_trim = DM_left_n3[delta:delta+left_img.shape[0], delta:delta+left_img.shape[1]]
DM_right_n3_trim = DM_right_n3[delta:delta+right_img.shape[0], delta:delta+right_img.shape[1]]

In [13]:
windowSize=9
#delta = int((windowSize-1)/2)
#xpad = ((delta,delta),(delta,delta))
#DM = np.pad(DisparityMatrix,xpad,mode='constant',constant_values=0)
DM_left_n9 = getDisparityMatrix(refImage=left_img,secondImage=right_img,wDim=windowSize,wLimit=75,l2r=False)
DM_right_n9 = getDisparityMatrix(refImage=right_img,secondImage=left_img,wDim=windowSize,wLimit=75,l2r=True)

In [14]:
DM_left_n9_trim = DM_left_n9[delta:delta+left_img.shape[0], delta:delta+left_img.shape[1]]
DM_right_n9_trim = DM_right_n9[delta:delta+right_img.shape[0], delta:delta+right_img.shape[1]]

Second Segment:Back propogation for consistency check

In [15]:
def getConsistencyCheckedDM(fixedDisparityMatrix,varDisparityMatrix,l2r=True):
    consistencyCheck = np.zeros(fixedDisparityMatrix.shape,dtype=np.int_)
    matchCount = 0
    for i in range(0,fixedDisparityMatrix.shape[0]):
        for j in range(0,fixedDisparityMatrix.shape[1]):
            #match each pixel in fixedImg with the var
            #newPos in varImg= yPos+disp
            if(l2r==True):
                yPosVar = j + fixedDisparityMatrix[i,j]
                if(yPosVar>=fixedDisparityMatrix.shape[1]):
                    consistencyCheck[i,j]=0
                else:
                    if(fixedDisparityMatrix[i,j]==varDisparityMatrix[i,yPosVar]):
                        #match found - copy the value of fixed into new matric
                        consistencyCheck[i,j]=fixedDisparityMatrix[i,j]
                        matchCount+=1
                    else:
                        consistencyCheck[i,j]=0
            else:
                yPosVar = j- fixedDisparityMatrix[i,j]
                if(yPosVar<0):
                    consistencyCheck[i,j]=0
                else:
                    if(fixedDisparityMatrix[i,j]==varDisparityMatrix[i,yPosVar]):
                        #match found - copy the value of fixed into new matric
                        consistencyCheck[i,j]=fixedDisparityMatrix[i,j]
                        matchCount+=1
                    else:
                        consistencyCheck[i,j]=0
    return (consistencyCheck,matchCount)

In [16]:
(CCl2r_n3,m1) = getConsistencyCheckedDM(fixedDisparityMatrix=DM_right_n3_trim,varDisparityMatrix=DM_left_n3_trim,l2r=True)
(CCr2l_n3,m2) = getConsistencyCheckedDM(fixedDisparityMatrix=DM_left_n3_trim,varDisparityMatrix=DM_right_n3_trim,l2r=False)

In [18]:
(CCl2r_n9,m1) = getConsistencyCheckedDM(fixedDisparityMatrix=DM_right_n9_trim,varDisparityMatrix=DM_left_n9_trim,l2r=True)
(CCr2l_n9,m2) = getConsistencyCheckedDM(fixedDisparityMatrix=DM_left_n9_trim,varDisparityMatrix=DM_right_n9_trim,l2r=False)

In [36]:
cv2.imwrite('view5_disp_w3x3.png',CCl2r_n3)
cv2.imwrite('view1_disp_w3x3.png',CCr2l_n3)
cv2.imwrite('view5_disp_w9x9.png',CCl2r_n9)
cv2.imwrite('view1_disp_w9x9.png',CCr2l_n9)
cv2.imwrite('',DM_left_n3_trim)

True

Third Segment:Calculate MSE

In [19]:
def getMse(imA,imB):
    SE=[]
    nonZeroIndexA = np.nonzero(imA)
    iterNum = len(nonZeroIndexA[0])
    for i in range(0,iterNum):
        xPos = nonZeroIndexA[0][i]
        yPos = nonZeroIndexA[1][i]
        diff = imA[xPos,yPos] - imB[xPos,yPos]
        SE.append(diff)
    diffSq = np.square(SE)
    diffSq = diffSq.flatten()
    MSE = np.sum(diffSq) / (imB.shape[0] * imB.shape[1])
    return MSE

In [20]:
#calculate MSE for window sizes 3x3, 9x9 and DM before and after backpropogation correction

In [21]:
mse_3x3_l = getMse(imA=DM_left_n3_trim,imB=left_depth)
mse_3x3_r = getMse(imA=DM_right_n3_trim,imB=right_depth)

In [22]:
mseCM_3x3_r = getMse(imA=CCl2r_n3,imB=right_depth)
mseCM_3x3_l = getMse(imA=CCr2l_n3,imB=left_depth)

In [23]:
mseCM_3x3_l

39

In [24]:
mseCM_3x3_r

40

In [27]:
mse_3x3_l

295

In [28]:
mse_3x3_r

211

In [ ]:
# 9x9

In [30]:
mse_9x9_l = getMse(imA=DM_left_n9_trim,imB=left_depth)
mse_9x9_r = getMse(imA=DM_right_n9_trim,imB=right_depth)

In [31]:
mseCM_9x9_r = getMse(imA=CCl2r_n9,imB=right_depth)
mseCM_9x9_l = getMse(imA=CCr2l_n9,imB=left_depth)

In [32]:
mseCM_9x9_l

28

In [33]:
mse_9x9_l

255

In [34]:
mseCM_9x9_r

30

In [35]:
mse_9x9_r

182